# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
import numpy as np
import matplotlib.pyplot as plt
import os, random
import json
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms, models
from collections import OrderedDict
from torch.utils.data import DataLoader
from PIL import Image
import time
from workspace_utils import active_session

print(os.getcwd())

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = '../aipnd-project/flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'
print("Train Directory")
#os.listdir(train_dir)


In [ ]:
print("Valid Directory")
#os.listdir(valid_dir)


In [ ]:
print("Test Directory")
#os.listdir(test_dir)

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets

train_transforms = transforms.Compose([ transforms.RandomHorizontalFlip(),
                                        transforms.RandomResizedCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
    
validation_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

# Load the train image dataset.
train_image_datasets = datasets.ImageFolder(train_dir, transform = train_transforms)


# Load the dataloaders using train image dataset.
train_dataloaders = DataLoader(dataset = train_image_datasets, batch_size = 64, shuffle=True)

# Load the dataloaders for validation image dataset.
val_image_datasets = datasets.ImageFolder(valid_dir, transform=validation_transforms)


# Validation dataloaders
val_dataloaders = DataLoader(dataset = val_image_datasets, batch_size = 32)

# Test Datasets
test_image_datasets = datasets.ImageFolder(test_dir, transform=test_transforms)

# Test dataloaders
test_dataloaders = DataLoader(dataset = test_image_datasets, batch_size = 32)



image_datasets = [train_image_datasets, val_image_datasets, test_image_datasets]
dataloaders = [train_dataloaders, val_dataloaders, test_dataloaders]

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
print(cat_to_name)    

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:

from workspace_utils import active_session
   
with active_session():
    
    # TODO: Build and train your network with Vgg16
    #vgg16_model = models.vgg16(pretrained=True)
    # Trying vgg19 model
    vgg19_model = models.vgg19(pretrained=True)
    #print(vgg16_model)
    
    #Freeze the model parameters.
    for param in vgg19_model.parameters():
        
        param.requires_grad = False
    
    
    classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 2048)),
                                           ('relu', nn.ReLU()),
                                           ('Dropout', nn.Dropout(0.5)),
                                           ('fc2', nn.Linear(2048,102)),
                                           ('output', nn.LogSoftmax(dim=1))]))

    
    #print(classifier)
    vgg19_model.classifier = classifier

    #print(vgg16_model)

    nn_entropy = nn.CrossEntropyLoss()
    #nn_entropy = nn.NLLLoss()

    nn_optimizer = optim.SGD(vgg19_model.classifier.parameters(), lr=0.002)
    #nn_optimizer = optim.Adam(vgg19_model.classifier.parameters(), lr=0.002)
    
    # Find the device whether it is cpu or GPU.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    
    #Move the model to GPU.
    vgg19_model.to(device)
    
    running_loss = 0
    accuracy = 0
    epochs = 1
    
    print("Start Training")

    start = time.time()

    # Run training and validation phases for each epochs. 
    for e in range(epochs):
       
        train_mode = 0
        valid_mode = 1
    
        for mode in [train_mode, valid_mode]:   
            
            if mode == train_mode:
                
                vgg19_model.train()
            else:
                vgg19_model.eval()
            
            pass_count = 0
        
            for data in dataloaders[mode]:
                
                pass_count += 1
                
                inputs, labels = data
                #print(labels)
                
                #Move the input and labels to GPU or CPU.
                inputs, labels = inputs.to(device), labels.to(device)
                
                #initialize to zero gradient.
                nn_optimizer.zero_grad()
            
                # Forward Pass
                output = vgg19_model.forward(inputs)
                loss = nn_entropy(output, labels)
            
                # Backward Pass only in case of training mode.
                if mode == train_mode:
                    loss.backward()
                    nn_optimizer.step()                
                
                # Loss accumulation.
                running_loss += loss.item()
                
                # Get probablities of the predicted class.
                ps = torch.exp(output)
            
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy = torch.mean(equals.type(torch.FloatTensor)).item()
                

            if mode == train_mode:
                print("\nEpoch_Number: {}/{} ".format(e+1, epochs))
                print("\nTraining Loss: {:.4f}  ".format(running_loss/pass_count))
                
            else:
                print("Validation Loss: {:.4f}  ".format(running_loss/pass_count))
                print("Accuracy: {:.4f}".format(accuracy))

            running_loss = 0

    time_elapsed = time.time() - start
    
    print("\nTime Taken for completing training.: {:.0f}m {:.0f}s".format(time_elapsed//60, time_elapsed % 60))


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [1]:
# TODO: Do validation on the test set
vgg19_model.eval()
accuracy = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
pass_count = 0

for data in test_dataloaders:
    pass_count += 1
    images, labels = data
    
    images, labels = images.to(device), labels.to(device)

    output = vgg19_model.forward(images)
    ps = torch.exp(output)
    
    top_p, top_class = ps.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print("Testing Accuracy: {:.4f}".format(accuracy/pass_count))

NameError: name 'vgg19_model' is not defined

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
vgg19_model.class_to_idx = image_datasets[0].class_to_idx

checkpoint = {'input_size': 25088,
              'output_size': 102,
              'arch': 'vgg19',
              'learning_rate': 0.002,
              'batch_size': 64,
              'classifier' : classifier,
              'epochs': epochs,
              'optimizer': nn_optimizer.state_dict(),
              'state_dict': vgg19_model.state_dict(),
              'class_to_idx': vgg19_model.class_to_idx}

torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filename):
    checkpoint = torch.load(filename)
    learning_rate = checkpoint['learning_rate']
    model = getattr(torchvision.models, checkpoint['arch'])(pretrained=True)
    model.classifier = checkpoint['classifier']
    model.epochs = checkpoint['epochs']
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    optimizer.load_state_dict(checkpoint['optimizer'])
        
    return model, optimizer

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    
    im = Image.open(image)
    im = im.resize((256,256))
    value = 0.5*(256-224)
    im = im.crop((value,value,256-value,256-value))
    im = np.array(im)/255

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    im = (im - mean) / std

    return im.transpose(2,0,1)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    if ax is None:
        fig, ax = plt.subplots()
    
    image = image.transpose((1, 2, 0))
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax



## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, device, topk=5):
    '''
    Predict the class (or classes) of an image using a trained deep learning model.
    '''

    #print(device)

    # if the device is GPU or CPU.
    if str(device) == "cuda":
        # Move model parameters to the GPU
        print("Number of GPUs: {}".format(torch.cuda.device_count()))
        print("Device name:", torch.cuda.get_device_name(torch.cuda.device_count()-1))

    model.to(device)

    # turn off dropout
    model.eval()

    # The image
    image = process_image(image_path)
    # Tranfer to tensor
    image = torch.from_numpy(np.array([image])).float()
    # Transfer the images to device.
    image = image.to(device)
    # pass the images in forward pass.
    output = model.forward(image)
    # Get the probabliites by taking the log of the data.
    probabilities = torch.exp(output).data

    # Getting probability and index.
    prob = torch.topk(probabilities, topk)[0].tolist()[0]
    print(prob)
    index = torch.topk(probabilities, topk)[1].tolist()[0]
    print(index)

    ind = []
    for i in range(len(model.class_to_idx.items())):
        ind.append(list(model.class_to_idx.items())[i][0])

    #print(ind)

    # transfer index to label
    label = []
    for i in range(5):
        label.append(ind[index[i]])
    print(label)

    return prob, label

In [ ]:
img = random.choice(os.listdir('./flowers/test/10/'))
img_path = './flowers/test/10/' + img
with  Image.open(img_path) as image:
    plt.imshow(image)
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
    
prob, classes = predict(img_path, vgg19_model, device)
print(prob)
print(classes)
print([cat_to_name[x] for x in classes])

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
prob, classes = predict(img_path, vgg19_model)
max_index = np.argmax(prob)
max_probability = prob[max_index]
label = classes[max_index]

fig = plt.figure(figsize=(10,10))
ax1 = plt.subplot2grid((15,9), (0,0), colspan=9, rowspan=9)
ax2 = plt.subplot2grid((15,9), (9,2), colspan=5, rowspan=5)

image = Image.open(img_path)
ax1.axis('off')
ax1.set_title(cat_to_name[label])
ax1.imshow(image)

top_labels = []
for cl in classes:
    top_labels.append(cat_to_name[cl])
    
y_pos = np.arange(5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(labels)
ax2.set_xlabel('Probability')
ax2.invert_yaxis()
ax2.barh(y_pos, prob, xerr=0, align='center', color='blue')

plt.show()